In [ ]:
#1: Get ticker code and CIK number with api for SEC FORM 4
# Print out FROM 4 tesla
import requests
import json
from datetime import datetime, timedelta

# Configuration
edgar_submissions_api = "https://data.sec.gov/submissions/CIK0001318605.json"  # Tesla CIK
date_range_days = 30  # To capture filings from 05/06/2025 to 06/05/2025

def search_form_4_api(date_range_days=30):
    """Search EDGAR for Form 4 filings via the Submissions API."""
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Accept": "application/json"
    }
    end_date = datetime.now()
    start_date = end_date - timedelta(days=date_range_days)
    
    try:
        start_time = time.time()
        response = requests.get(edgar_submissions_api, headers=headers, timeout=5)
        response.raise_for_status()
        print(f"API request took {time.time() - start_time:.2f} seconds")
        
        # Parse the JSON response
        data = response.json()
        filings = data.get("filings", {}).get("recent", {})
        if not filings:
            print("No filings found in the API response.")
            return None
        
        # Extract filing details
        accession_numbers = filings.get("accessionNumber", [])
        filing_dates = filings.get("filingDate", [])
        form_types = filings.get("form", [])
        
        # Filter filings by date range and form type
        filtered_filings = []
        for i in range(len(filing_dates)):
            filing_date = datetime.strptime(filing_dates[i], "%Y-%m-%d")
            if start_date.date() <= filing_date.date() <= end_date.date():
                if form_types[i] == "4":  # Only include Form 4 filings
                    filtered_filings.append({
                        "form": form_types[i],
                        "filing_date": filing_dates[i],
                        "accession_number": accession_numbers[i],
                        "filing_url": f"https://www.sec.gov/Archives/edgar/data/1318605/{accession_numbers[i].replace('-', '')}/{accession_numbers[i]}-index.htm"
                    })
        
        print(f"\nSearch Parameters:")
        print(f"CIK: 0001318605")
        print(f"Date Range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
        print(f"Forms: 4")
        
        print("\nAPI Response (Filtered Filings):")
        print(json.dumps(filtered_filings, indent=2))
        return filtered_filings
    
    except requests.RequestException as e:
        print(f"Error searching EDGAR API: {e}")
        return None

if __name__ == "__main__":
    search_form_4_api()

API request took 0.59 seconds

Search Parameters:
CIK: 0001318605
Date Range: 2025-05-06 to 2025-06-05
Forms: 4

API Response (Filtered Filings):
[
  {
    "form": "4",
    "filing_date": "2025-06-04",
    "accession_number": "0001104659-25-056561",
    "filing_url": "https://www.sec.gov/Archives/edgar/data/1318605/000110465925056561/0001104659-25-056561-index.htm"
  },
  {
    "form": "4",
    "filing_date": "2025-05-29",
    "accession_number": "0001104659-25-054372",
    "filing_url": "https://www.sec.gov/Archives/edgar/data/1318605/000110465925054372/0001104659-25-054372-index.htm"
  },
  {
    "form": "4",
    "filing_date": "2025-05-29",
    "accession_number": "0001104659-25-054371",
    "filing_url": "https://www.sec.gov/Archives/edgar/data/1318605/000110465925054371/0001104659-25-054371-index.htm"
  },
  {
    "form": "4",
    "filing_date": "2025-05-20",
    "accession_number": "0001771340-25-000009",
    "filing_url": "https://www.sec.gov/Archives/edgar/data/1318605/00017713

In [ ]:
#2: Next step go further action of informaiton from filing_url
import requests
import json
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

# Configuration
edgar_submissions_api = "https://data.sec.gov/submissions/CIK0001318605.json"  # Tesla CIK
date_range_days = 30  # To capture filings from 05/06/2025 to 06/05/2025

def fetch_form_4_details(filing_url, filing_date):
    """Fetch and inspect Form 4 filing details from the given URL."""
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Accept": "text/html, application/xml"
    }

    try:
        # Step 1: Fetch the index page
        print(f"\nFetching index page for Filing Date: {filing_date}")
        print(f"URL: {filing_url}")
        response = requests.get(filing_url, headers=headers, timeout=5)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Print the document table from the index page
        table = soup.find("table", class_="tableFile")
        if not table:
            print(f"No document table found in index page for {filing_date}")
            print("Index Page Content (snippet):")
            print(response.text[:1000])
            return None
        
        print("\nDocument Table from Index Page:")
        print(table.prettify())
        
        # Step 2: Find both Form 4 document links (HTML and XML)
        html_link = None
        xml_link = None
        for row in table.find_all("tr")[1:]:  # Skip header row
            cols = row.find_all("td")
            if len(cols) >= 4:
                description = cols[1].text.strip().upper()
                doc_type = cols[3].text.strip()
                link = cols[2].find("a")
                if ("OWNERSHIP DOCUMENT" in description or "PRIMARY DOCUMENT" in description or doc_type == "4") and link:
                    href = link["href"]
                    if "xslF345X05" in href and href.endswith((".xml", ".htm", ".html")):
                        html_link = "https://www.sec.gov" + href
                    elif href.endswith(".xml") and "xslF345X05" not in href:
                        xml_link = "https://www.sec.gov" + href
        
        # Step 3: Inspect both links
        if xml_link:
            print(f"\nFound Potential XML Link: {xml_link}")
            time.sleep(0.2)
            response = requests.get(xml_link, headers=headers, timeout=5)
            response.raise_for_status()
            print(f"\nRaw Content of Potential XML Document for {filing_date}:")
            print(response.text[:2000])  # Print more characters to see full structure
            
            # Attempt to parse as XML
            try:
                root = ET.fromstring(response.text)
                namespace = {"ns": root.tag.split("}")[0].strip("{")}  # Extract namespace
                
                # Extract key details with error handling
                issuer_elem = root.find(".//ns:issuer/ns:issuerName", namespace)
                issuer_name = issuer_elem.text.strip() if issuer_elem is not None else "Unknown"
                
                reporting_owner_elem = root.find(".//ns:reportingOwner/ns:reportingOwnerId/ns:rptOwnerName", namespace)
                reporting_owner = reporting_owner_elem.text.strip() if reporting_owner_elem is not None else "Unknown"
                
                transactions = []
                # Non-derivative transactions
                for transaction in root.findall(".//ns:nonDerivativeTable/ns:nonDerivativeTransaction", namespace):
                    trans_date_elem = transaction.find("ns:transactionDate/ns:value", namespace)
                    trans_date = trans_date_elem.text.strip() if trans_date_elem is not None else "Unknown"
                    
                    trans_code_elem = transaction.find("ns:transactionCoding/ns:transactionCode", namespace)
                    trans_code = trans_code_elem.text.strip() if trans_code_elem is not None else "Unknown"
                    
                    shares_elem = transaction.find("ns:transactionAmounts/ns:transactionShares/ns:value", namespace)
                    shares = shares_elem.text.strip() if shares_elem is not None else "0"
                    
                    price_elem = transaction.find("ns:transactionAmounts/ns:transactionPricePerShare/ns:value", namespace)
                    price = price_elem.text.strip() if price_elem is not None else "N/A"
                    
                    ownership_elem = transaction.find("ns:postTransactionAmounts/ns:ownershipNature/ns:directOrIndirectOwnership/ns:value", namespace)
                    ownership = ownership_elem.text.strip() if ownership_elem is not None else "Unknown"
                    
                    transactions.append({
                        "transaction_date": trans_date,
                        "transaction_code": trans_code,
                        "shares": shares,
                        "price_per_share": price,
                        "ownership_type": ownership
                    })
                
                # Print the details
                print(f"\nForm 4 Details for Filing Date: {filing_date}")
                print(f"Issuer: {issuer_name}")
                print(f"Reporting Owner: {reporting_owner}")
                print("Transactions:")
                if transactions:
                    for trans in transactions:
                        print(f"  - Transaction Date: {trans['transaction_date']}")
                        print(f"    Transaction Code: {trans['transaction_code']} (P=Purchase, S=Sale)")
                        print(f"    Shares: {trans['shares']}")
                        print(f"    Price per Share: {trans['price_per_share']}")
                        print(f"    Ownership Type: {trans['ownership_type']} (D=Direct, I=Indirect)")
                else:
                    print("No transactions found in XML.")
                
                return transactions
            
            except ET.ParseError as e:
                print(f"Error parsing XML for {filing_date}: {e}")
        
        if html_link:
            print(f"\nFound HTML Link: {html_link}")
            time.sleep(0.2)
            response = requests.get(html_link, headers=headers, timeout=5)
            response.raise_for_status()
            print(f"\nRaw Content of HTML Document for {filing_date} (first 2000 characters):")
            print(response.text[:2000])
            
            # Parse as HTML
            soup = BeautifulSoup(response.text, "html.parser")
            
            # Extract issuer and reporting owner
            issuer_name = soup.find(text=lambda t: "Issuer Name" in t)
            issuer_name = issuer_name.find_next("span", class_="FormData").text.strip() if issuer_name else "Unknown"
            reporting_owner = soup.find(text=lambda t: "Reporting Owner Name" in t)
            reporting_owner = reporting_owner.find_next("span", class_="FormData").text.strip() if reporting_owner else "Unknown"
            
            # Find non-derivative transactions table
            transactions = []
            non_deriv_table = soup.find("table", summary=lambda s: s and "Non-Derivative" in s)
            if non_deriv_table:
                for row in non_deriv_table.find_all("tr")[1:]:  # Skip header
                    cols = row.find_all("td")
                    if len(cols) >= 7:
                        trans_date = cols[0].text.strip()
                        trans_code = cols[1].text.strip()
                        shares = cols[3].text.strip()
                        price = cols[4].text.strip() or "N/A"
                        ownership = cols[6].text.strip()
                        
                        transactions.append({
                            "transaction_date": trans_date,
                            "transaction_code": trans_code,
                            "shares": shares,
                            "price_per_share": price,
                            "ownership_type": ownership
                        })
            
            # Print the details
            print(f"\nForm 4 Details for Filing Date: {filing_date}")
            print(f"Issuer: {issuer_name}")
            print(f"Reporting Owner: {reporting_owner}")
            print("Transactions:")
            if transactions:
                for trans in transactions:
                    print(f"  - Transaction Date: {trans['transaction_date']}")
                    print(f"    Transaction Code: {trans['transaction_code']} (P=Purchase, S=Sale)")
                    print(f"    Shares: {trans['shares']}")
                    print(f"    Price per Share: {trans['price_per_share']}")
                    print(f"    Ownership Type: {trans['ownership_type']} (D=Direct, I=Indirect)")
            else:
                print("No transactions found in HTML.")
            
            return transactions
        
        print(f"No suitable Form 4 document link found for {filing_date}")
        return None
    
    except requests.RequestException as e:
        print(f"Error fetching Form 4 details for {filing_date}: {e}")
        return None

def search_form_4_api(date_range_days=30):
    """Search EDGAR for Form 4 filings via the Submissions API and inspect each filing URL."""
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Accept": "application/json"
    }
    end_date = datetime.now()
    start_date = end_date - timedelta(days=date_range_days)
    
    try:
        start_time = time.time()
        response = requests.get(edgar_submissions_api, headers=headers, timeout=5)
        response.raise_for_status()
        print(f"API request took {time.time() - start_time:.2f} seconds")
        
        # Parse the JSON response
        data = response.json()
        filings = data.get("filings", {}).get("recent", {})
        if not filings:
            print("No filings found in the API response.")
            return None
        
        # Extract filing details
        accession_numbers = filings.get("accessionNumber", [])
        filing_dates = filings.get("filingDate", [])
        form_types = filings.get("form", [])
        
        # Filter filings by date range and form type
        filtered_filings = []
        for i in range(len(filing_dates)):
            filing_date = datetime.strptime(filing_dates[i], "%Y-%m-%d")
            if start_date.date() <= filing_date.date() <= end_date.date():
                if form_types[i] == "4":
                    filtered_filings.append({
                        "form": form_types[i],
                        "filing_date": filing_dates[i],
                        "accession_number": accession_numbers[i],
                        "filing_url": f"https://www.sec.gov/Archives/edgar/data/1318605/{accession_numbers[i].replace('-', '')}/{accession_numbers[i]}-index.htm"
                    })
        
        print(f"\nSearch Parameters:")
        print(f"CIK: 0001318605")
        print(f"Date Range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
        print(f"Forms: 4")
        
        print("\nAPI Response (Filtered Filings):")
        print(json.dumps(filtered_filings, indent=2))
        
        # Inspect each filing URL
        for filing in filtered_filings:
            fetch_form_4_details(filing["filing_url"], filing["filing_date"])
            time.sleep(0.2)  # Delay to avoid rate limiting
        
        return filtered_filings
    
    except requests.RequestException as e:
        print(f"Error searching EDGAR API: {e}")
        return None

if __name__ == "__main__":
    search_form_4_api()

API request took 0.57 seconds

Search Parameters:
CIK: 0001318605
Date Range: 2025-05-06 to 2025-06-05
Forms: 4

API Response (Filtered Filings):
[
  {
    "form": "4",
    "filing_date": "2025-06-04",
    "accession_number": "0001104659-25-056561",
    "filing_url": "https://www.sec.gov/Archives/edgar/data/1318605/000110465925056561/0001104659-25-056561-index.htm"
  },
  {
    "form": "4",
    "filing_date": "2025-05-29",
    "accession_number": "0001104659-25-054372",
    "filing_url": "https://www.sec.gov/Archives/edgar/data/1318605/000110465925054372/0001104659-25-054372-index.htm"
  },
  {
    "form": "4",
    "filing_date": "2025-05-29",
    "accession_number": "0001104659-25-054371",
    "filing_url": "https://www.sec.gov/Archives/edgar/data/1318605/000110465925054371/0001104659-25-054371-index.htm"
  },
  {
    "form": "4",
    "filing_date": "2025-05-20",
    "accession_number": "0001771340-25-000009",
    "filing_url": "https://www.sec.gov/Archives/edgar/data/1318605/00017713

In [ ]:
# 3: fileing_url to get deeper info about which xml to parse
# Print out all information from FORM 4
import requests
import json
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

# Configuration
EDGAR_SUBMISSIONS_API = "https://data.sec.gov/submissions/CIK0001318605.json"  # Tesla CIK
DATE_RANGE_DAYS = 30  # To capture filings from 05/06/2025 to 06/05/2025

def fetch_form_4_details(filing_url, filing_date):
    """
    Fetch and parse Form 4 filing details from the given URL (XML preferred, HTML fallback).
    Returns a list of transaction dicts.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Accept": "application/xml, text/html"
    }

    try:
        print(f"\nFetching index page for Filing Date: {filing_date}")
        print(f"URL: {filing_url}")
        response = requests.get(filing_url, headers=headers, timeout=5)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        # --- STEP 1: Locate the XML and HTML links in the tableFile ---
        table = soup.find("table", class_="tableFile")
        if not table:
            print(f"No document table found in index page for {filing_date}")
            return []

        xml_link = None
        html_link = None
        for row in table.find_all("tr")[1:]:  # Skip header
            cols = row.find_all("td")
            if len(cols) >= 4:
                desc = cols[1].get_text(strip=True).upper()
                doctype = cols[3].get_text(strip=True)
                link_tag = cols[2].find("a")
                if link_tag and doctype == "4" and ("OWNERSHIP DOCUMENT" in desc or "PRIMARY DOCUMENT" in desc):
                    href = link_tag["href"]
                    if href.endswith(".xml"):
                        xml_link = "https://www.sec.gov" + href
                    elif href.endswith((".htm", ".html")) and "xslF345X05" in href:
                        html_link = "https://www.sec.gov" + href

        transactions = []

        # --- STEP 2: Try to download & parse XML if it exists ---
        if xml_link:
            print(f"\nFound Potential XML Link: {xml_link}")
            time.sleep(0.2)
            xml_resp = requests.get(xml_link, headers=headers, timeout=5)
            xml_resp.raise_for_status()
            xml_text = xml_resp.text

            print(f"\nRaw Content of XML Document for {filing_date} (first 500 chars):")
            print(xml_text[:500])

            try:
                root = ET.fromstring(xml_text)

                # Detect namespace (if any)
                if root.tag.startswith("{"):
                    ns_uri = root.tag.split("}")[0].strip("{")
                    ns = {"ns": ns_uri}
                    use_ns = True
                else:
                    ns = {}
                    use_ns = False

                # Extract issuerName and reportingOwnerName
                if use_ns:
                    issuer_elem = root.find(".//ns:issuer/ns:issuerName", ns)
                    owner_elem  = root.find(".//ns:reportingOwner/ns:reportingOwnerId/ns:rptOwnerName", ns)
                    txn_xpath   = ".//ns:nonDerivativeTable/ns:nonDerivativeTransaction"
                else:
                    issuer_elem = root.find(".//issuer/issuerName")
                    owner_elem  = root.find(".//reportingOwner/reportingOwnerId/rptOwnerName")
                    txn_xpath   = ".//nonDerivativeTable/nonDerivativeTransaction"

                issuer_name = issuer_elem.text.strip() if issuer_elem is not None else "Unknown"
                reporting_owner = owner_elem.text.strip() if owner_elem is not None else "Unknown"

                print(f"\nParsed via XML for Filing Date {filing_date}:")
                print(f"Issuer: {issuer_name}")
                print(f"Reporting Owner: {reporting_owner}")

                # Extract each non‐derivative transaction
                for txn_node in root.findall(txn_xpath, ns if use_ns else None):
                    if use_ns:
                        date_elem      = txn_node.find("ns:transactionDate/ns:value", ns)
                        code_elem      = txn_node.find("ns:transactionCoding/ns:transactionCode", ns)
                        shares_elem    = txn_node.find("ns:transactionAmounts/ns:transactionShares/ns:value", ns)
                        price_elem     = txn_node.find("ns:transactionAmounts/ns:transactionPricePerShare/ns:value", ns)
                        ownership_elem = txn_node.find(
                            "ns:postTransactionAmounts/ns:ownershipNature/ns:directOrIndirectOwnership/ns:value", ns
                        )
                    else:
                        date_elem      = txn_node.find("transactionDate/value")
                        code_elem      = txn_node.find("transactionCoding/transactionCode")
                        shares_elem    = txn_node.find("transactionAmounts/transactionShares/value")
                        price_elem     = txn_node.find("transactionAmounts/transactionPricePerShare/value")
                        ownership_elem = txn_node.find(
                            "postTransactionAmounts/ownershipNature/directOrIndirectOwnership/value"
                        )

                    trans_date = date_elem.text.strip() if date_elem is not None else "Unknown"
                    trans_code = code_elem.text.strip() if code_elem is not None else "Unknown"
                    shares     = shares_elem.text.strip() if shares_elem is not None else "0"
                    price      = price_elem.text.strip() if price_elem is not None else "N/A"
                    ownership  = ownership_elem.text.strip() if ownership_elem is not None else "Unknown"

                    transactions.append({
                        "transaction_date": trans_date,
                        "transaction_code": trans_code,
                        "shares": shares,
                        "price_per_share": price,
                        "ownership_type": ownership
                    })

                if transactions:
                    print("\nTransactions found in XML:")
                    for tx in transactions:
                        print(f"  • Date: {tx['transaction_date']}  |  Code: {tx['transaction_code']}  "
                              f"| Shares: {tx['shares']}  | Price: {tx['price_per_share']}  | Ownership: {tx['ownership_type']}")
                else:
                    print("No non‐derivative transactions found in XML.")

                return transactions

            except ET.ParseError as e:
                print(f"XML parsing error for {filing_date}: {e}")
                # Fall back to HTML parsing below

        # --- STEP 3: Fallback to HTML parsing if XML was missing or yielded nothing ---
        if html_link:
            print(f"\nFalling back to HTML parsing. Found HTML Link: {html_link}")
            time.sleep(0.2)
            html_resp = requests.get(html_link, headers=headers, timeout=5)
            html_resp.raise_for_status()
            html_soup = BeautifulSoup(html_resp.text, "html.parser")

            # Extract issuer and reporting owner from HTML
            issuer_label = html_soup.find(text=lambda t: t and "Issuer Name" in t)
            if issuer_label:
                issuer_name = issuer_label.find_next("span", class_="FormData").get_text(strip=True)
            else:
                issuer_name = "Unknown"

            owner_label = html_soup.find(text=lambda t: t and "Reporting Owner Name" in t)
            if owner_label:
                reporting_owner = owner_label.find_next("span", class_="FormData").get_text(strip=True)
            else:
                reporting_owner = "Unknown"

            print(f"\nParsed via HTML for Filing Date {filing_date}:")
            print(f"Issuer: {issuer_name}")
            print(f"Reporting Owner: {reporting_owner}")

            # Extract the Non‐Derivative Transactions table
            non_deriv_table = html_soup.find("table", summary=lambda s: s and "Non‐Derivative" in s)
            if non_deriv_table:
                rows = non_deriv_table.find_all("tr")[1:]  # Skip header row
                for row in rows:
                    cols = row.find_all("td")
                    if len(cols) >= 7:
                        trans_date = cols[0].get_text(strip=True)
                        trans_code = cols[1].get_text(strip=True)
                        shares     = cols[3].get_text(strip=True) or "0"
                        price      = cols[4].get_text(strip=True) or "N/A"
                        ownership  = cols[6].get_text(strip=True) or "Unknown"

                        transactions.append({
                            "transaction_date": trans_date,
                            "transaction_code": trans_code,
                            "shares": shares,
                            "price_per_share": price,
                            "ownership_type": ownership
                        })

                if transactions:
                    print("\nTransactions found in HTML:")
                    for tx in transactions:
                        print(f"  • Date: {tx['transaction_date']}  |  Code: {tx['transaction_code']}  "
                              f"| Shares: {tx['shares']}  | Price: {tx['price_per_share']}  | Ownership: {tx['ownership_type']}")
                else:
                    print("No non‐derivative transactions found in HTML.")
            else:
                print("No Non‐Derivative table found in HTML.")

            return transactions

        # --- STEP 4: No XML or HTML to parse ---
        print(f"No XML or HTML data to parse for {filing_date}. Returning empty list.")
        return []

    except requests.RequestException as e:
        print(f"Error fetching Form 4 details for {filing_date}: {e}")
        return []

def search_form_4_api(date_range_days=DATE_RANGE_DAYS):
    """
    Search EDGAR for Form 4 filings via the Submissions API and inspect each filing URL.
    Returns a list of filtered filings (with parsed transactions printed along the way).
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Accept": "application/json"
    }
    end_date = datetime.now()
    start_date = end_date - timedelta(days=date_range_days)

    try:
        start_time = time.time()
        response = requests.get(EDGAR_SUBMISSIONS_API, headers=headers, timeout=5)
        response.raise_for_status()
        print(f"API request took {time.time() - start_time:.2f} seconds")

        data = response.json()
        filings = data.get("filings", {}).get("recent", {})
        if not filings:
            print("No filings found in the API response.")
            return []

        accession_numbers = filings.get("accessionNumber", [])
        filing_dates      = filings.get("filingDate", [])
        form_types        = filings.get("form", [])

        filtered_filings = []
        for i in range(len(filing_dates)):
            filing_date_obj = datetime.strptime(filing_dates[i], "%Y-%m-%d")
            if start_date.date() <= filing_date_obj.date() <= end_date.date():
                if form_types[i] == "4":
                    acc_no = accession_numbers[i]
                    fil_url = (
                        f"https://www.sec.gov/Archives/edgar/data/1318605/"
                        f"{acc_no.replace('-', '')}/{acc_no}-index.htm"
                    )
                    filtered_filings.append({
                        "form": form_types[i],
                        "filing_date": filing_dates[i],
                        "accession_number": acc_no,
                        "filing_url": fil_url
                    })

        print(f"\nSearch Parameters:")
        print(f"CIK: 0001318605")
        print(f"Date Range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
        print(f"Forms: 4")

        print("\nAPI Response (Filtered Filings):")
        print(json.dumps(filtered_filings, indent=2))

        # Inspect each filing URL
        for filing in filtered_filings:
            fetch_form_4_details(filing["filing_url"], filing["filing_date"])
            time.sleep(0.2)  # Delay to avoid rate limiting

        return filtered_filings

    except requests.RequestException as e:
        print(f"Error searching EDGAR API: {e}")
        return []

if __name__ == "__main__":
    search_form_4_api()

API request took 0.47 seconds

Search Parameters:
CIK: 0001318605
Date Range: 2025-05-06 to 2025-06-05
Forms: 4

API Response (Filtered Filings):
[
  {
    "form": "4",
    "filing_date": "2025-06-04",
    "accession_number": "0001104659-25-056561",
    "filing_url": "https://www.sec.gov/Archives/edgar/data/1318605/000110465925056561/0001104659-25-056561-index.htm"
  },
  {
    "form": "4",
    "filing_date": "2025-05-29",
    "accession_number": "0001104659-25-054372",
    "filing_url": "https://www.sec.gov/Archives/edgar/data/1318605/000110465925054372/0001104659-25-054372-index.htm"
  },
  {
    "form": "4",
    "filing_date": "2025-05-29",
    "accession_number": "0001104659-25-054371",
    "filing_url": "https://www.sec.gov/Archives/edgar/data/1318605/000110465925054371/0001104659-25-054371-index.htm"
  },
  {
    "form": "4",
    "filing_date": "2025-05-20",
    "accession_number": "0001771340-25-000009",
    "filing_url": "https://www.sec.gov/Archives/edgar/data/1318605/00017713

In [ ]:
#4: Print out content FORM 4 and check what information to parse
import requests

def print_xml_contents():
    url = "https://www.sec.gov/Archives/edgar/data/1318605/000110465925056561/tm2517124-1_4seq1.xml"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # will raise an error if the request failed

    # Print the raw XML content
    print(response.text)

if __name__ == "__main__":
    print_xml_contents()


<?xml version="1.0"?>
<ownershipDocument>

    <schemaVersion>X0508</schemaVersion>

    <documentType>4</documentType>

    <periodOfReport>2025-06-02</periodOfReport>

    <notSubjectToSection16>0</notSubjectToSection16>

    <issuer>
        <issuerCik>0001318605</issuerCik>
        <issuerName>Tesla, Inc.</issuerName>
        <issuerTradingSymbol>TSLA</issuerTradingSymbol>
    </issuer>

    <reportingOwner>
        <reportingOwnerId>
            <rptOwnerCik>0001771340</rptOwnerCik>
            <rptOwnerName>Taneja Vaibhav</rptOwnerName>
        </reportingOwnerId>
        <reportingOwnerAddress>
            <rptOwnerStreet1>C/O TESLA, INC.</rptOwnerStreet1>
            <rptOwnerStreet2>1 TESLA ROAD</rptOwnerStreet2>
            <rptOwnerCity>AUSTIN</rptOwnerCity>
            <rptOwnerState>TX</rptOwnerState>
            <rptOwnerZipCode>78725</rptOwnerZipCode>
            <rptOwnerStateDescription></rptOwnerStateDescription>
        </reportingOwnerAddress>
        <reportingOwne

In [ ]:
#5: Based on url to parse all information from data
import requests
import xml.etree.ElementTree as ET
import pandas as pd

def fetch_and_parse_form4_full():
    # --- 1. Fetch the raw XML from the SEC URL ---
    url = "https://www.sec.gov/Archives/edgar/data/1318605/000110465925056561/tm2517124-1_4seq1.xml"
    headers = {
        # Per SEC requirements, include a clear User-Agent with your contact info if you automate heavily
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "(compatible; MyScript/1.0; +your-email@example.com)"
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an error if the request failed (e.g., 403, 404)

    # Store the fetched XML into a string
    xml_text = response.text

    # --- 2. Parse the XML into an ElementTree ---
    root = ET.fromstring(xml_text)

    # --- 3. Extract issuer information (same for all transactions) ---
    issuer = root.find("issuer")
    issuer_cik = issuer.find("issuerCik").text if issuer is not None and issuer.find("issuerCik") is not None else ""
    issuer_name = issuer.find("issuerName").text if issuer is not None and issuer.find("issuerName") is not None else ""
    issuer_symbol = issuer.find("issuerTradingSymbol").text if issuer is not None and issuer.find("issuerTradingSymbol") is not None else ""

    # --- 4. Extract reportingOwner information (same for all transactions) ---
    rep_owner = root.find("reportingOwner")
    # reportingOwnerId subelement
    ro_id = rep_owner.find("reportingOwnerId") if rep_owner is not None else None
    reporting_owner_cik = ro_id.find("rptOwnerCik").text if ro_id is not None and ro_id.find("rptOwnerCik") is not None else ""
    reporting_owner_name = ro_id.find("rptOwnerName").text if ro_id is not None and ro_id.find("rptOwnerName") is not None else ""
    # officerTitle under reportingOwnerRelationship
    officer_title = rep_owner.find("reportingOwnerRelationship/officerTitle").text \
                    if rep_owner is not None and rep_owner.find("reportingOwnerRelationship/officerTitle") is not None \
                    else ""

    # Prepare list to collect all transaction records
    records = []

    # --- 5. Parse non-derivative transactions ---
    for txn in root.findall(".//nonDerivativeTransaction"):
        def get_text(path):
            node = txn.find(path)
            return node.text.strip() if node is not None and node.text else ""

        footnote_ids = [fn.attrib["id"] for fn in txn.findall("transactionCoding/footnoteId")]

        records.append({
            # Common fields
            "Transaction Type":           "Non-Derivative",
            "Issuer CIK":                 issuer_cik,
            "Issuer Name":                issuer_name,
            "Issuer Symbol":              issuer_symbol,
            "Reporting Owner CIK":        reporting_owner_cik,
            "Reporting Owner Name":       reporting_owner_name,
            "Officer Title":              officer_title,
            # Non-Derivative-specific fields
            "Security Title":             get_text("securityTitle/value"),
            "Transaction Date":           get_text("transactionDate/value"),
            "Deemed Execution Date":      get_text("deemedExecutionDate/value"),
            "Form Type":                  get_text("transactionCoding/transactionFormType"),
            "Transaction Code":           get_text("transactionCoding/transactionCode"),
            "Equity Swap Involved":       get_text("transactionCoding/equitySwapInvolved"),
            "Transaction Shares":         get_text("transactionAmounts/transactionShares/value"),
            "Price per Share":            get_text("transactionAmounts/transactionPricePerShare/value"),
            "Acquired/Disposed Code":     get_text("transactionAmounts/transactionAcquiredDisposedCode/value"),
            "Shares Owned After":         get_text("postTransactionAmounts/sharesOwnedFollowingTransaction/value"),
            "Ownership Type":             get_text("ownershipNature/directOrIndirectOwnership/value"),
            "Footnote IDs":               ", ".join(footnote_ids),
            # Derivative fields (blank for non-derivative)
            "Conversion/Exercise Price":  "",
            "Expiration Date":            "",
            "Underlying Security Title":  "",
            "Underlying Security Shares": ""
        })

    # --- 6. Parse derivative transactions ---
    for dtxn in root.findall(".//derivativeTransaction"):
        def get_text(path):
            node = dtxn.find(path)
            return node.text.strip() if node is not None and node.text else ""

        footnote_ids = [fn.attrib["id"] for fn in dtxn.findall("transactionCoding/footnoteId")]

        underlying_title = get_text("underlyingSecurity/underlyingSecurityTitle/value")
        underlying_shares = get_text("underlyingSecurity/underlyingSecurityShares/value")

        records.append({
            # Common fields
            "Transaction Type":           "Derivative",
            "Issuer CIK":                 issuer_cik,
            "Issuer Name":                issuer_name,
            "Issuer Symbol":              issuer_symbol,
            "Reporting Owner CIK":        reporting_owner_cik,
            "Reporting Owner Name":       reporting_owner_name,
            "Officer Title":              officer_title,
            # Derivative-specific fields (reuse some field names from non-derivative for convenience)
            "Security Title":             get_text("securityTitle/value"),
            "Transaction Date":           get_text("transactionDate/value"),
            "Deemed Execution Date":      get_text("deemedExecutionDate/value"),
            "Form Type":                  get_text("transactionCoding/transactionFormType"),
            "Transaction Code":           get_text("transactionCoding/transactionCode"),
            "Equity Swap Involved":       get_text("transactionCoding/equitySwapInvolved"),
            "Transaction Shares":         get_text("transactionAmounts/transactionShares/value"),
            "Price per Share":            get_text("transactionAmounts/transactionPricePerShare/value"),
            "Acquired/Disposed Code":     get_text("transactionAmounts/transactionAcquiredDisposedCode/value"),
            "Shares Owned After":         get_text("postTransactionAmounts/sharesOwnedFollowingTransaction/value"),
            "Ownership Type":             get_text("ownershipNature/directOrIndirectOwnership/value"),
            "Footnote IDs":               ", ".join(footnote_ids),
            # Extra derivative fields
            "Conversion/Exercise Price":  get_text("conversionOrExercisePrice/value"),
            "Expiration Date":            get_text("expirationDate/value"),
            "Underlying Security Title":  underlying_title,
            "Underlying Security Shares": underlying_shares
        })

    # --- 7. Convert list of dicts into a DataFrame ---
    df = pd.DataFrame(records)
    return df

if __name__ == "__main__":
    df = fetch_and_parse_form4_full()
    print(df)

   Transaction Type  Issuer CIK  Issuer Name Issuer Symbol  \
0    Non-Derivative  0001318605  Tesla, Inc.          TSLA   
1    Non-Derivative  0001318605  Tesla, Inc.          TSLA   
2    Non-Derivative  0001318605  Tesla, Inc.          TSLA   
3    Non-Derivative  0001318605  Tesla, Inc.          TSLA   
4    Non-Derivative  0001318605  Tesla, Inc.          TSLA   
5    Non-Derivative  0001318605  Tesla, Inc.          TSLA   
6    Non-Derivative  0001318605  Tesla, Inc.          TSLA   
7    Non-Derivative  0001318605  Tesla, Inc.          TSLA   
8    Non-Derivative  0001318605  Tesla, Inc.          TSLA   
9    Non-Derivative  0001318605  Tesla, Inc.          TSLA   
10   Non-Derivative  0001318605  Tesla, Inc.          TSLA   
11   Non-Derivative  0001318605  Tesla, Inc.          TSLA   
12   Non-Derivative  0001318605  Tesla, Inc.          TSLA   
13       Derivative  0001318605  Tesla, Inc.          TSLA   
14       Derivative  0001318605  Tesla, Inc.          TSLA   

   Repo

In [ ]:
# Final: Parse SEC full version data including insider stock actions, shares, options, filing & issue date, option prices & shares & expiration date
import requests
import json
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import pandas as pd
import os
from datetime import datetime

# Configuration
CIK = "0001318605"  # change as needed
EDGAR_SUBMISSIONS_API = f"https://data.sec.gov/submissions/CIK{CIK}.json"
############################################################################################
# cik_list = [
#     "0001318605",  # Tesla
#     "0000320193",  # Apple
#     "0001652044",  # Alphabet
# ]
# for CIK in cik_list:
#     EDGAR_SUBMISSIONS_API = f"https://data.sec.gov/submissions/CIK{CIK}.json"
#     print(f"\nProcessing CIK={CIK}  →  {EDGAR_SUBMISSIONS_API}")
############################################################################################
DATE_RANGE_DAYS = 30  # Capture filings from 30 days ago to today

def get_xml_link_from_index(index_url):
    """
    Given a filing index URL (the "-index.htm" page), scrape the tableFile to find
    the *raw* XML link for the Form 4 (skip any xslF345X05 paths). Return the full XML URL,
    or None if not found.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }
    resp = requests.get(index_url, headers=headers, timeout=5)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "html.parser")
    table = soup.find("table", class_="tableFile")
    if not table:
        return None

    for row in table.find_all("tr")[1:]:  # skip header row
        cols = row.find_all("td")
        if len(cols) < 4:
            continue

        desc = cols[1].get_text(strip=True).upper()
        doctype = cols[3].get_text(strip=True)
        link_tag = cols[2].find("a")
        if (
            link_tag
            and doctype == "4"
            and ("OWNERSHIP DOCUMENT" in desc or "PRIMARY DOCUMENT" in desc)
        ):
            href = link_tag["href"]
            # select only the raw XML (no xslF345X05)
            if href.endswith(".xml") and "xslF345X05" not in href:
                return "https://www.sec.gov" + href
    return None

def parse_form4_xml(xml_url):
    """
    Given a Form 4 raw XML URL, fetch and parse it. Return a list of dictionaries,
    one per transaction (non-derivative & derivative), with issuer, owner, and all fields.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "(compatible; MyScript/1.0; +your-email@example.com)"
    }
    resp = requests.get(xml_url, headers=headers, timeout=5)
    resp.raise_for_status()
    xml_text = resp.text
    root = ET.fromstring(xml_text)

    # Detect namespace
    if root.tag.startswith("{"):
        ns_uri = root.tag.split("}")[0].strip("{")
        ns = {"ns": ns_uri}
        use_ns = True
    else:
        ns = {}
        use_ns = False

    # Helper to extract text given a path, with or without namespace
    def get_text(node, path):
        if use_ns:
            elem = node.find(path, ns)
        else:
            elem = node.find(path)
        return elem.text.strip() if (elem is not None and elem.text) else ""

    # --- Extract issuer info ---
    if use_ns:
        issuer_cik    = get_text(root, "ns:issuer/ns:issuerCik")
        issuer_name   = get_text(root, "ns:issuer/ns:issuerName")
        issuer_symbol = get_text(root, "ns:issuer/ns:issuerTradingSymbol")
    else:
        issuer_cik    = get_text(root, "issuer/issuerCik")
        issuer_name   = get_text(root, "issuer/issuerName")
        issuer_symbol = get_text(root, "issuer/issuerTradingSymbol")

    # --- Extract reporting owner info ---
    if use_ns:
        reporting_owner_cik  = get_text(root, ".//ns:reportingOwner/ns:reportingOwnerId/ns:rptOwnerCik")
        reporting_owner_name = get_text(root, ".//ns:reportingOwner/ns:reportingOwnerId/ns:rptOwnerName")
        officer_title        = get_text(root, ".//ns:reportingOwner/ns:reportingOwnerRelationship/ns:officerTitle")
    else:
        reporting_owner_cik  = get_text(root, ".//reportingOwner/reportingOwnerId/rptOwnerCik")
        reporting_owner_name = get_text(root, ".//reportingOwner/reportingOwnerId/rptOwnerName")
        officer_title        = get_text(root, ".//reportingOwner/reportingOwnerRelationship/officerTitle")

    records = []

    # --- Parse non-derivative transactions ---
    if use_ns:
        non_deriv_xpath = ".//ns:nonDerivativeTable/ns:nonDerivativeTransaction"
    else:
        non_deriv_xpath = ".//nonDerivativeTable/nonDerivativeTransaction"

    for txn in root.findall(non_deriv_xpath, ns if use_ns else None):
        if use_ns:
            footnodes = txn.findall("ns:transactionCoding/ns:footnoteId", ns)
        else:
            footnodes = txn.findall("transactionCoding/footnoteId")
        footnote_ids = [fn.attrib.get("id", "") for fn in footnodes]

        records.append({
            "Filing XML URL":            xml_url,
            "Transaction Type":          "Non-Derivative",
            "Issuer CIK":                issuer_cik,
            "Issuer Name":               issuer_name,
            "Issuer Symbol":             issuer_symbol,
            "Reporting Owner CIK":       reporting_owner_cik,
            "Reporting Owner Name":      reporting_owner_name,
            "Officer Title":             officer_title,
            "Security Title":            get_text(txn, "ns:securityTitle/ns:value" if use_ns else "securityTitle/value"),
            "Transaction Date":          get_text(txn, "ns:transactionDate/ns:value" if use_ns else "transactionDate/value"),
            "Deemed Execution Date":     get_text(txn, "ns:deemedExecutionDate" if use_ns else "deemedExecutionDate"),
            "Form Type":                 get_text(txn, "ns:transactionCoding/ns:transactionFormType" if use_ns else "transactionCoding/transactionFormType"),
            "Transaction Code":          get_text(txn, "ns:transactionCoding/ns:transactionCode" if use_ns else "transactionCoding/transactionCode"),
            "Equity Swap Involved":      get_text(txn, "ns:transactionCoding/ns:equitySwapInvolved" if use_ns else "transactionCoding/equitySwapInvolved"),
            "Transaction Shares":        get_text(txn, "ns:transactionAmounts/ns:transactionShares/ns:value" if use_ns else "transactionAmounts/transactionShares/value"),
            "Price per Share":           get_text(txn, "ns:transactionAmounts/ns:transactionPricePerShare/ns:value" if use_ns else "transactionAmounts/transactionPricePerShare/value"),
            "Acquired/Disposed Code":    get_text(txn, "ns:transactionAmounts/ns:transactionAcquiredDisposedCode/ns:value" if use_ns else "transactionAmounts/transactionAcquiredDisposedCode/value"),
            "Shares Owned After":        get_text(txn, "ns:postTransactionAmounts/ns:sharesOwnedFollowingTransaction/ns:value" if use_ns else "postTransactionAmounts/sharesOwnedFollowingTransaction/value"),
            "Ownership Type":            get_text(txn, "ns:ownershipNature/ns:directOrIndirectOwnership/ns:value" if use_ns else "ownershipNature/directOrIndirectOwnership/value"),
            "Footnote IDs":              ", ".join(footnote_ids),
            # Derivative-only fields left blank here
            "Conversion/Exercise Price":  "",
            "Expiration Date":            "",
            "Underlying Security Title":  "",
            "Underlying Security Shares": ""
        })

    # --- Parse derivative transactions ---
    if use_ns:
        deriv_xpath = ".//ns:derivativeTable/ns:derivativeTransaction"
    else:
        deriv_xpath = ".//derivativeTable/derivativeTransaction"

    for dtxn in root.findall(deriv_xpath, ns if use_ns else None):
        if use_ns:
            footnodes = dtxn.findall("ns:transactionCoding/ns:footnoteId", ns)
        else:
            footnodes = dtxn.findall("transactionCoding/footnoteId")
        footnote_ids = [fn.attrib.get("id", "") for fn in footnodes]

        underlying_title  = get_text(dtxn, "ns:underlyingSecurity/ns:underlyingSecurityTitle/ns:value" if use_ns else "underlyingSecurity/underlyingSecurityTitle/value")
        underlying_shares = get_text(dtxn, "ns:underlyingSecurity/ns:underlyingSecurityShares/ns:value" if use_ns else "underlyingSecurity/underlyingSecurityShares/value")

        records.append({
            "Filing XML URL":            xml_url,
            "Transaction Type":          "Derivative",
            "Issuer CIK":                issuer_cik,
            "Issuer Name":               issuer_name,
            "Issuer Symbol":             issuer_symbol,
            "Reporting Owner CIK":       reporting_owner_cik,
            "Reporting Owner Name":      reporting_owner_name,
            "Officer Title":             officer_title,
            "Security Title":            get_text(dtxn, "ns:securityTitle/ns:value" if use_ns else "securityTitle/value"),
            "Transaction Date":          get_text(dtxn, "ns:transactionDate/ns:value" if use_ns else "transactionDate/value"),
            "Deemed Execution Date":     get_text(dtxn, "ns:deemedExecutionDate" if use_ns else "deemedExecutionDate"),
            "Form Type":                 get_text(dtxn, "ns:transactionCoding/ns:transactionFormType" if use_ns else "transactionCoding/transactionFormType"),
            "Transaction Code":          get_text(dtxn, "ns:transactionCoding/ns:transactionCode" if use_ns else "transactionCoding/transactionCode"),
            "Equity Swap Involved":      get_text(dtxn, "ns:transactionCoding/ns:equitySwapInvolved" if use_ns else "transactionCoding/equitySwapInvolved"),
            "Transaction Shares":        get_text(dtxn, "ns:transactionAmounts/ns:transactionShares/ns:value" if use_ns else "transactionAmounts/transactionShares/value"),
            "Price per Share":           get_text(dtxn, "ns:transactionAmounts/ns:transactionPricePerShare/ns:value" if use_ns else "transactionAmounts/transactionPricePerShare/value"),
            "Acquired/Disposed Code":    get_text(dtxn, "ns:transactionAmounts/ns:transactionAcquiredDisposedCode/ns:value" if use_ns else "transactionAmounts/transactionAcquiredDisposedCode/value"),
            "Shares Owned After":        get_text(dtxn, "ns:postTransactionAmounts/ns:sharesOwnedFollowingTransaction/ns:value" if use_ns else "postTransactionAmounts/sharesOwnedFollowingTransaction/value"),
            "Ownership Type":            get_text(dtxn, "ns:ownershipNature/ns:directOrIndirectOwnership/ns:value" if use_ns else "ownershipNature/directOrIndirectOwnership/value"),
            "Footnote IDs":              ", ".join(footnote_ids),
            # Now fill derivative-specific fields
            "Conversion/Exercise Price": get_text(dtxn, "ns:conversionOrExercisePrice/ns:value" if use_ns else "conversionOrExercisePrice/value"),
            "Expiration Date":           get_text(dtxn, "ns:expirationDate/ns:value" if use_ns else "expirationDate/value"),
            "Underlying Security Title": underlying_title,
            "Underlying Security Shares": underlying_shares
        })

    return records

def search_form_4_api_and_collect():
    """
    Search EDGAR for Form 4 filings via the Submissions API, then collect all XML links
    for those Form 4 filings into a list. Return a list of tuples:
    (filing_date, index_url, xml_url).
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Accept": "application/json"
    }
    end_date = datetime.now()
    start_date = end_date - timedelta(days=DATE_RANGE_DAYS)

    response = requests.get(EDGAR_SUBMISSIONS_API, headers=headers, timeout=5)
    response.raise_for_status()
    data = response.json()
    filings = data.get("filings", {}).get("recent", {})
    accession_numbers = filings.get("accessionNumber", [])
    filing_dates      = filings.get("filingDate", [])
    form_types        = filings.get("form", [])

    tuples = []
    for i in range(len(filing_dates)):
        filing_date_obj = datetime.strptime(filing_dates[i], "%Y-%m-%d")
        if start_date.date() <= filing_date_obj.date() <= end_date.date():
            if form_types[i] == "4":
                acc_no = accession_numbers[i]
                index_url = (
                    f"https://www.sec.gov/Archives/edgar/data/1318605/"
                    f"{acc_no.replace('-', '')}/{acc_no}-index.htm"
                )
                tuples.append((filing_dates[i], index_url))
    
    results = []
    for filing_date, idx_url in tuples:
        xml_link = get_xml_link_from_index(idx_url)
        if xml_link:
            results.append((filing_date, idx_url, xml_link))
    return results

if __name__ == "__main__":
    # Step A: Collect all (filing_date, index_url, xml_url) tuples
    collected = search_form_4_api_and_collect()

    today_str = datetime.now().strftime("%Y%m%d")

    print("All Filing Tuples (Date, Index URL, Raw XML URL):")
    for filing_date, idx_url, xml_url in collected:
        print(f"  • {filing_date} | {idx_url} | {xml_url}")

    # Step B: Parse each raw XML URL, adding Filing Date to each record
    all_records = []
    for filing_date, idx_url, xml_url in collected:
        time.sleep(0.2)  # avoid rate-limiting
        try:
            records = parse_form4_xml(xml_url)
            # Insert "Filing Date" in each record dict
            for rec in records:
                rec["Filing Date"] = filing_date
            all_records.extend(records)
        except Exception as e:
            print(f"Failed to parse {xml_url}: {e}")

    # Step C: Convert combined records into a DataFrame and display
    df = pd.DataFrame(all_records)
    # Reorder columns so "Filing Date" appears near front
    cols = ["Filing Date", "Filing XML URL", "Transaction Type", "Issuer CIK", "Issuer Name",
            "Issuer Symbol", "Reporting Owner CIK", "Reporting Owner Name", "Officer Title"] \
           + [c for c in df.columns if c not in {
               "Filing Date", "Filing XML URL", "Transaction Type", "Issuer CIK",
               "Issuer Name", "Issuer Symbol", "Reporting Owner CIK", "Reporting Owner Name", "Officer Title"
           }]
    df = df[cols]

    print("\nCombined DataFrame of All Transactions:")
    print(df.to_string(index=False))
    # 1. Derive TICKER from the DataFrame’s "Issuer Symbol" column
    if not df.empty and "Issuer Symbol" in df.columns:
        ticker = df["Issuer Symbol"].iloc[0]
    else:
        ticker = "UNKNOWN"
    
    today_str = datetime.now().strftime("%Y%m%d")

    filename = f"form4_{ticker}_{CIK}_{today_str}_last{DATE_RANGE_DAYS}d.csv"

    # (Optional) ensure an output directory
    os.makedirs("output", exist_ok=True)
    full_path = os.path.join("output", filename)

    df.to_csv(full_path, index=False)
    print(f"\nSaved combined data to {full_path}")


All Filing Tuples (Date, Index URL, Raw XML URL):
  • 2025-06-04 | https://www.sec.gov/Archives/edgar/data/1318605/000110465925056561/0001104659-25-056561-index.htm | https://www.sec.gov/Archives/edgar/data/1318605/000110465925056561/tm2517124-1_4seq1.xml
  • 2025-05-29 | https://www.sec.gov/Archives/edgar/data/1318605/000110465925054372/0001104659-25-054372-index.htm | https://www.sec.gov/Archives/edgar/data/1318605/000110465925054372/tm2516319-2_4seq1.xml
  • 2025-05-29 | https://www.sec.gov/Archives/edgar/data/1318605/000110465925054371/0001104659-25-054371-index.htm | https://www.sec.gov/Archives/edgar/data/1318605/000110465925054371/tm2516319-1_4seq1.xml
  • 2025-05-20 | https://www.sec.gov/Archives/edgar/data/1318605/000177134025000009/0001771340-25-000009-index.htm | https://www.sec.gov/Archives/edgar/data/1318605/000177134025000009/edgardoc.xml
  • 2025-05-13 | https://www.sec.gov/Archives/edgar/data/1318605/000177134025000008/0001771340-25-000008-index.htm | https://www.sec.go